In [1]:
using Plots, Interact
include("AutomDiff_V2.jl")
import AD: Dual, xdual
pyplot(size = (500, 500))

Plots.PyPlotBackend()

In [2]:
mapins(x, M, S, R) = (R*x^2)/((R - 1)*x^2/M + S + x)

mapins (generic function with 1 method)

In [3]:
"""
    ciclosestables!(xx, f, nit, nout, cc)

Esta función itera el mapeo `f`, de una variable, `nit+nout` veces, 
usando como condición inicial `x0=0`; los últimos `nout` iterados 
actualizan al vector `xx` que tiene longitud `nout`. `cc` es el valor
del parámetro del mapeo `f`. El mapeo `f` debe ser definido de 
tal manera que `f(x0,cc)` tenga sentido. La idea es los últimos 
`nout` iterados reflejen los ciclos estables del mapeo `f`. 
"""
function ciclosestables!(xx, f, nit, nout, cc)
    @assert nit > 0 && nout > 0
    
    # Primeros nit iterados
    x0 = 1000.0
    for it = 1:nit
        x0 = f(x0, cc)
    end
    
    # Se guardan los siguientes nout iterados
    for it = 1:nout
        x0 = f(x0, cc)
        @inbounds xx[it] = x0
    end
    
    nothing
end
"""
    diagbifurc(f, nit, nout, crange)

Itera el mapeo `f` `nit+nout` veces y regresa una matriz
cuya columna `i` tiene los últimos `nout` iterados del mapeo
para el valor del parámetro del mapeo `crange[i]`.

La función `f` debe ser definida de tal manera que `f(x0, c)` 
tenga sentido.
"""
function diagbifurc(f, nit, nout, crange)
    xx = Vector{Float64}(nout)
    ff = Array{Float64,2}(nout, length(crange))
    
    for ic in eachindex(crange)
        c = crange[ic]
        ciclosestables!(xx, f, nit, nout, c)
        ff[:,ic] = xx
    end
    
    return ff
end

doc"""
    Lyapunov(y, F)

Dado un punto un arreglo que contenga todas las iteraciones bajo $F$ de $x_0$, calcula 
$\frac{1}{n} \sum_{i=1}^n \log\Big| \, f'(x_i)\, \Big|.$
"""
function Lyapunov(y, F::Function)
    n = length(y)
    l = 0
   
    for i in 1:n
        x = abs((F(xdual(y[i]))).der)
        l = l + log(x)
       
    end
    l/n
end

doc"""
    iterado(F,x0,n)

Dado un punto $x_0$, esta funció guarda a todas la iteraciones bajo $F$ en un arreglo.
"""
function iterado(F::Function,x0::Float64,n::Int)
    y = Float64[x0] ##El primer elemento es x_0
    x = x0
    for i in 1:n
        x=F(x) #El valor de x cambia con cada iteración
        push!(y,x) #Este nuevo valor se guarda en el arreglo
    end
    y
end



doc"""
mapeo_liap_all(crange::FloatRange, Qc::Function, x0::Float64, n = 100000)

Calcula el exponente de Lyapunov para un mapeo paramétrico $Q_c = Q_c(x, c)$ en un rango de parámetros `crange`, un
valor inicial `x0` y `n` iteraciones.
"""
function mapeo_liap_all(crange::FloatRange, Qc::Function, x0::Float64, n = 100_000)
    mapeo = Float64[]
  
    for i in crange
        Qci(x) = Qc(x,i) 
        y = iterado(Qci,x0,n)
        l = Lyapunov(y, Qci)
        if abs(l)<Inf #En algunos puntos labda diverge, por lo que si tiende a infinito, se la asigna el valor-2.5
            push!(mapeo,l)      
        else
            push!(mapeo,-2.5)       
        end
      
    end
    mapeo
end

mapeo_liap_all

In [ ]:
@manipulate for M in 0.1:0.1:10, R in 1.1:0.1:10
    m1(x, s) = mapins(x, M, s, R)
    srange = 0:200.0
    map1 = mapeo_liap_all(srange, m1, 1000.0)
    ff = diagbifurc(m1, 1000, 256, srange); 
    cc = ones(size(ff)[1])*srange';

# Esto cambia las matrices en vectores; ayuda un poco para los dibujos
    ff = reshape(ff, size(ff)[1]*size(ff)[2]);
    cc = reshape(cc, size(ff));
    scatter(cc, ff, markersize= 0.05)
    ylims!(0, 10)
end

In [4]:
1*785*0.5/3600

0.10902777777777778

In [5]:
M1 = 1:1
R1 = 2:1/2^3:100
zeros(length(M1), length(R1))

1×785 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [14]:
M1 = 1:10
R1 = 2:100
Mat1 = zeros(length(M1), length(R1))
K = length(M1)*length(R1)
for j in eachindex(M1), k in eachindex(R1)
    M, R = M1[j], R1[k]
    m1(x, s) = mapins(x, M, s, R)
    srange = 0:1000.0
    #map1 = mapeo_liap_all(srange, m1, 1000.0)
    ff = diagbifurc(m1, 1000, 256, srange); 
    cc = ones(size(ff)[1])*srange';
    if mod(j*k/K*100, 10) == 0
        @show "Llevo $(j*k/K*100)%"
    end
# Esto cambia las matrices en vectores; ayuda un poco para los dibujos
    ff = reshape(ff, size(ff)[1]*size(ff)[2]);
    cc = reshape(cc, size(ff));
    for l in eachindex(ff)
        if ff[l] == 0
            Mat1[j, k] = cc[l]
        end
    end
end
    

"Llevo $(((j * k) / K) * 100)%" = "Llevo 10.0%"
"Llevo $(((j * k) / K) * 100)%" = "Llevo 20.0%"
"Llevo $(((j * k) / K) * 100)%" = "Llevo 10.0%"
"Llevo $(((j * k) / K) * 100)%" = "Llevo 20.0%"
"Llevo $(((j * k) / K) * 100)%" = "Llevo 30.0%"
"Llevo $(((j * k) / K) * 100)%" = "Llevo 40.0%"


LoadError: LoadError: InterruptException:
while loading In[14], in expression starting on line 5

In [10]:
using PyPlot

In [13]:
Mat1

10×99 Array{Float64,2}:
 1000.0  1000.0  1000.0  1000.0  1000.0  …  1000.0  1000.0  1000.0  1000.0
 1000.0  1000.0  1000.0  1000.0  1000.0     1000.0  1000.0  1000.0  1000.0
 1000.0  1000.0  1000.0  1000.0  1000.0     1000.0  1000.0  1000.0  1000.0
 1000.0  1000.0  1000.0  1000.0  1000.0     1000.0  1000.0  1000.0  1000.0
 1000.0  1000.0  1000.0  1000.0  1000.0     1000.0  1000.0  1000.0  1000.0
 1000.0  1000.0  1000.0  1000.0  1000.0  …  1000.0  1000.0  1000.0  1000.0
 1000.0  1000.0  1000.0  1000.0  1000.0     1000.0  1000.0  1000.0  1000.0
 1000.0  1000.0  1000.0  1000.0  1000.0     1000.0  1000.0  1000.0  1000.0
 1000.0  1000.0  1000.0  1000.0  1000.0     1000.0  1000.0  1000.0  1000.0
 1000.0  1000.0  1000.0  1000.0  1000.0     1000.0  1000.0  1000.0  1000.0

In [12]:
imshow(Mat1)

show()

In [ ]:
for j in eachindex(ff)
    if ff[j] == 0
        return cc[j]
    end
end

In [ ]:
scatter(cc, ff, markersize= 0.05)

In [ ]:
scatter(srange, map1)
    #xlims!(0,200)
    #ylims!(0, 100)

In [ ]:
a1

In [ ]:
a1 = maximum(map1)
for j in eachindex(map1)
    if map1[j] == a1
        return srange[j]
    end
end